In [ ]:
from ipywidgets import widgets
import io
#import librosa
import xgboost 
from xgboost import XGBClassifier
#import numpy as np
import torch, torchaudio
import pickle

In [ ]:
model = XGBClassifier().load_model("Audio_XBG_model.pickle")

<h1><center>Harry Potter or Star Wars

This web app based on machine learning is created to the classify a tune either as "Harry Potter" or "Star Wars". To do so, click the upload button below, and select the the audio file. 

In [ ]:
lbl_greet = widgets.Label('Select the Audio File')

btn_upload = widgets.FileUpload()

out_pl = widgets.Output()

lbl_pred = widgets.Label()

In [ ]:
def on_data_change(change):
    lbl_pred.value = ''
    
    
    for name, file_info in btn_upload.value.items():
        #audio, sample_rate = librosa.load(io.BytesIO(file_info['content']), res_type='kaiser_fast')
        signal, sr = torchaudio.load(io.BytesIO(file_info['content']))
    """
    #print('Uploaded audio')
    def mfcc_extractor(file_, sample_rate):
        mfccs_features = librosa.feature.mfcc(y=file_, sr=sample_rate, n_mfcc=64)
        mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
        return mfccs_scaled_features
    
    test_audio = mfcc_extractor(audio, sample_rate)
    test_audio.shape = -1,64
    """
    
    out_pl.clear_output()
    
    SAMPLE_RATE = 22050
    NFFT = 1024
    HLEN = 512
    NMEL = 64
    
    mel_spectogram = torchaudio.transforms.MelSpectrogram(
        sample_rate= SAMPLE_RATE,
        n_fft= NFFT,
        hop_length= HLEN,
        n_mels= NMEL)(signal)
    
    if sr != SAMPLE_RATE:
        resampler = torchaudio.transforms.Resample(sr, 22050)
        mel_spectogram_resampled = resampler(signal)
    
    if mel_spectogram_resampled.shape[0] > 1:
        mel_signal = torch.mean(mel_spectogram_resampled, dim=0, keepdim=True)
    
    if mel_signal.shape[1] > NMEL:
        mel_signal = mel_signal[:, :NMEL]
    
    class_mapping = [
    "Star Wars",
    "Harry Potter"
    ]
    
    # PREDICT 
    prediction_index = model.predict(mel_signal.tolist())
    prediction = class_mapping[prediction_index[0]]

    #print(f"Predicted: '{prediction}'")
    
    lbl_pred.value = f"Predicted: '{prediction}'"
    

In [ ]:
btn_upload.observe(on_data_change, names=['data'])

In [ ]:
display(lbl_greet, btn_upload, out_pl, lbl_pred)